In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import json
from collections import defaultdict
import os

In [11]:
# Main Data Imports
snapshot = pd.read_parquet('Data/snapshot_player_stats.parquet')
player_stats = pd.read_parquet('Data/player_game_stats.parquet')

In [10]:
snapshot.head(1)

,game_urn,player_urn,seq_idx,event_date_time,game_time,winning_team,player,team_urn,team,champion,...,vision_score,total_damage_dealt_champions,total_damage_taken,item_1_id,item_2_id,item_3_id,item_4_id,item_5_id,item_6_id,item_7_id
0,live:lol:riot:map:esportstmnt01-3373466,live:lol:riot:player:051c9431-16d0-3872-98c4-c...,16,2023-06-14 21:13:15.338000+00:00,138,None,Fudge,live:lol:riot:team:98767991877340524,C9,KSante,...,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
snapshot.columns

Index(['game_urn', 'player_urn', 'seq_idx', 'event_date_time', 'game_time',
       'winning_team', 'player', 'team_urn', 'team', 'champion', 'level',
       'alive', 'health', 'current_gold', 'total_gold', 'gold_per_second',
       'pos_x', 'pos_y', 'minions_killed', 'neutral_minions_killed',
       'neutral_minions_killed_your_jungle',
       'neutral_minions_killed_enemy_jungle', 'champs_killed', 'deaths',
       'assists', 'wards_placed', 'wards_killed', 'vision_score',
       'total_damage_dealt_champions', 'total_damage_taken', 'item_1_id',
       'item_2_id', 'item_3_id', 'item_4_id', 'item_5_id', 'item_6_id',
       'item_7_id'],
      dtype='object')

In [40]:
blabber_snapshots = snapshot[(snapshot['player'] == 'Blaber') & (snapshot['seq_idx'] <= 360)]

In [41]:
blabber_snapshots.shape

(10005, 37)

In [65]:
c9_team_urn = player_stats[player_stats['team'] == 'C9']['game_urn'].unique()
c9_team_urn

array(['live:lol:riot:map:esportstmnt01-3373466',
       'live:lol:riot:map:esportstmnt01-3375039',
       'live:lol:riot:map:esportstmnt01-3376646',
       'live:lol:riot:map:esportstmnt01-3408908',
       'live:lol:riot:map:esportstmnt01-3409908',
       'live:lol:riot:map:esportstmnt01-3410017',
       'live:lol:riot:map:esportstmnt01-3413353',
       'live:lol:riot:map:esportstmnt01-3414314',
       'live:lol:riot:map:esportstmnt01-3414318',
       'live:lol:riot:map:esportstmnt01-3416273',
       'live:lol:riot:map:esportstmnt01-3416274',
       'live:lol:riot:map:esportstmnt01-3417173',
       'live:lol:riot:map:esportstmnt01-3417177',
       'live:lol:riot:map:esportstmnt02-3207804',
       'live:lol:riot:map:esportstmnt02-3209643',
       'live:lol:riot:map:esportstmnt02-3210080',
       'live:lol:riot:map:esportstmnt02-3213623',
       'live:lol:riot:map:esportstmnt02-3214865',
       'live:lol:riot:map:esportstmnt02-3215315',
       'live:lol:riot:map:esportstmnt03-3176763',


In [70]:
player_stats.columns

Index(['game_urn', 'player_urn', 'team_urn', 'region', 'game_start_datetime',
       'best_of', 'curr_game', 'player', 'team', 'champion', 'side', 'role',
       'opp_player_urn', 'opp_player', 'opp_team_urn', 'opp_team',
       'opp_champion', 'result', 'kills', 'deaths', 'assists', 'double_kills',
       'triple_kills', 'quad_kills', 'penta_kills', 'first_blood',
       'avg_mins_between_kills', 'avg_mins_between_deaths', 'solo_kills',
       'iso_deaths', 'minion_kills', 'neutral_minion_kills',
       'neutral_minion_kills_team_jungle', 'neutral_minion_kills_enemy_jungle',
       'perc_in_mid_lane', 'num_recalls', 'total_damage_teamfights',
       'avg_damage_per_teamfight', 'dmg_to_champs_per_min',
       'dmg_shielded_on_team_per_min', 'dmg_healed_on_team_per_min',
       'total_gold', 'gold_at_15m', 'gold_diff_at_15m', 'wards_placed',
       'wards_killed', 'vision_score', 'jungle_prox', 'duo_prox', 'team_perid',
       'player_perid', 'opp_player_perid', 'opp_team_urn_perid', 'u

In [68]:
c9_team_urn = player_stats[player_stats['opp_team'] == 'C9']['match_urn'].unique()

KeyError: 'match_urn'

In [66]:
player_stats[player_stats['opp_team_urn'] == c9_team_urn]['game_urn'].unique()

ValueError: ('Lengths must match to compare', (570,), (29,))

In [59]:
blue_side_games = player_stats[player_stats['team_urn'] == c9_team_urn]['game_urn']
red_side_games = player_stats[player_stats['opp_team_urn'] == c9_team_urn]['game_urn']

In [52]:
red_side_games

,game_urn
0,live:lol:riot:map:esportstmnt01-3373466
6,live:lol:riot:map:esportstmnt01-3373466
7,live:lol:riot:map:esportstmnt01-3373466
8,live:lol:riot:map:esportstmnt01-3373466
9,live:lol:riot:map:esportstmnt01-3373466
...,...
550,live:lol:riot:map:esportstmnt03-3194842
551,live:lol:riot:map:esportstmnt03-3194842
552,live:lol:riot:map:esportstmnt03-3194842
553,live:lol:riot:map:esportstmnt03-3194842


In [51]:
blabber_snapshots['game_urn'].isin(red_side_games['game_urn'])

1          True
11         True
21         True
31         True
41         True
           ... 
3703936    True
3703946    True
3703956    True
3703966    True
3703976    True
Name: game_urn, Length: 10005, dtype: bool

In [75]:
blabber_snapshots.columns

Index(['game_urn', 'player_urn', 'seq_idx', 'event_date_time', 'game_time',
       'winning_team', 'player', 'team_urn', 'team', 'champion', 'level',
       'alive', 'health', 'current_gold', 'total_gold', 'gold_per_second',
       'pos_x', 'pos_y', 'minions_killed', 'neutral_minions_killed',
       'neutral_minions_killed_your_jungle',
       'neutral_minions_killed_enemy_jungle', 'champs_killed', 'deaths',
       'assists', 'wards_placed', 'wards_killed', 'vision_score',
       'total_damage_dealt_champions', 'total_damage_taken', 'item_1_id',
       'item_2_id', 'item_3_id', 'item_4_id', 'item_5_id', 'item_6_id',
       'item_7_id', 'side'],
      dtype='object')

In [44]:
blabber_snapshots['side'] = 'blue'
blabber_snapshots.loc[blabber_snapshots['game_urn'].isin(red_side_games['game_urn']), 'side'] = 'red'

C:\Users\Ethan\AppData\Local\Temp\ipykernel_22288\2093071840.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  blabber_snapshots['side'] = 'blue'


In [48]:
from ipywidgets import interactive, widgets

# Load the image
lol_map = plt.imread("minimap.png")

def generate_jungle_path_plot(champion, side, seq_idx, data):
    # Set the dimensions of the LoL map for proper overlay
    extent = [0, 16_000, 0, 16_000]  # values based on your provided dimensions
    
    # Filter data based on champion
    champ_data = data[data['champion'] == champion]
    
    # Filter data up to the given timestamp
    champ_data = champ_data[champ_data['seq_idx'] <= seq_idx]  # assuming timestamp is in minutes
    
    # Filter based on the selected side
    if side != 'both':
        champ_data = champ_data[champ_data['side'] == side]
    
    plt.figure(figsize=(12, 12))
    
    # Display the map image
    plt.imshow(lol_map, extent=extent)
    
    # Plot paths
    for side in champ_data['side'].unique():
        side_data = champ_data[champ_data['side'] == side]
        plt.plot(side_data['pos_x'], side_data['pos_y'], label=f"{champion} - {side}", color='red')
    
    plt.legend()
    plt.title(f"Jungle Path for {champion}")
    plt.xlabel('X Coordinate')
    plt.ylabel('Y Coordinate')
    plt.show()

def interactive_jungle_path(champion, side, timestamp):
    return generate_jungle_path_plot(champion, side, timestamp, blabber_snapshots)

# Unique champions in the dataset
champions = blabber_snapshots['champion'].unique()

# Set up the interactive element:
widget = interactive(interactive_jungle_path, 
                     champion=widgets.Dropdown(options=champions, description='Champion:'),
                     side=widgets.Dropdown(options=['blue', 'red', 'both'], value='both', description='Side:'),
                     timestamp=widgets.IntSlider(value=16, min=16, max=360, step=1, description='Seq Id:'))  # Adjust max time as needed

display(widget)


interactive(children=(Dropdown(description='Champion:', options=('Kindred', 'Khazix', 'Sejuani', 'Maokai', 'Re…